In [209]:
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import ipywidgets as widgets
from IPython.display import display
import os
import datetime

# LLM's
from dotenv import load_dotenv
import os

from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain import PromptTemplate

from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number

# Texto
import nltk
from langdetect import detect

In [2]:
load_dotenv()

True

In [3]:
openai_api_key= os.getenv('OPENAI_APIKEY')

# Carga datos totales:

In [39]:
ruta_datos = "C:\\Users\\regue\\Desktop\\Data Science Projects\\PROJECTS\\IT_Job_Spain_Project\\Datos\\datos_sin_procesar\\"

In [340]:
df_datos_finales = pd.read_csv(ruta_datos + 'datos_trabajos.csv')

In [41]:
df_datos_finales.head(3)

,titulo,empresa,fecha,herramientas,descripcion,ubicacion,presencialidad,funciones,jornada,experiencia,tipo_contrato,salario,beneficios,solicitudes,fecha_scrapeo,url,portal
0,data scientist,capgemini engineering,2024-01-04,"['aptitudes para hacer presentaciones', 'comun...",acerca del empleohybrid intelligence a capgemi...,barcelona,hibrido,NaN,jornada completa,0,no especificado,NaN,NaN,52,2024-01-25,https://www.linkedin.com/jobs/search/?currentj...,linkedin
1,data scientist,italki,2024-01-18,"['aws glue', 'aws lambda', 'aws sagemaker', 'a...",acerca del empleoabout the position italki is ...,madrid,hibrido,NaN,jornada completa,0,no especificado,NaN,NaN,56,2024-01-25,https://www.linkedin.com/jobs/search/?currentj...,linkedin
2,data scientist,gft technologies,2024-01-04,"['airflow', 'análisis de clústers', 'análisis ...",acerca del empleodescripcion del puesto que es...,madrid,hibrido,NaN,jornada completa,0,no especificado,NaN,NaN,35,2024-01-25,https://www.linkedin.com/jobs/search/?currentj...,linkedin


# Veamos la información faltante:

<span style="font-size:larger;">

Veamos cada una de las columnas:

</span>

In [149]:
df_datos_finales.isna().sum()

titulo               1
empresa              2
fecha                0
herramientas       540
descripcion          1
ubicacion         2149
presencialidad       0
funciones          802
jornada              0
experiencia          0
tipo_contrato        0
salario           1977
beneficios        4921
solicitudes          0
fecha_scrapeo        0
url               4034
portal               0
dtype: int64

<span style="font-size:larger;">

- Eliminar elemento 3513

</span>

In [43]:
df_datos_finales[df_datos_finales["titulo"].isna()]

,titulo,empresa,fecha,herramientas,descripcion,ubicacion,presencialidad,funciones,jornada,experiencia,tipo_contrato,salario,beneficios,solicitudes,fecha_scrapeo,url,portal
3513,NaN,NaN,2024-01-26,NaN,NaN,NaN,no especificado,NaN,no especificado,0,no especificado,NaN,NaN,0,2024-01-26,NaN,infojobs


# Busqueda de información faltante en las columnas ubicación y salario con LLM: ---- FUNCIONAL!

In [277]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-1106",
    temperature=0,
    max_tokens=2000,
    openai_api_key=openai_api_key
)

In [282]:
job_schema = Object(id="Job offer",
                    description="Job offer information.",

                    
                    attributes=[Text(id="ubicacion", description="Job Location."),
                                Number(id="salario",description="The salary or salary range offered for the job position.")
                                ],
                    
                    examples=[("En KPMG, líder en soluciones tecnológicas innovadoras, buscamos un Desarrollador de Software Senior para unirse a nuestro equipo en la sede de Madrid. Serás parte integral del equipo de desarrollo, contribuyendo a la creación de soluciones robustas y eficientes para nuestros clientes. Colaborar con equipos multifuncionales para comprender y abordar los requisitos del cliente. Experiencia comprobada en desarrollo de software, preferiblemente con enfoque en tecnologías como Java, Python o C#. Ambiente de trabajo dinámico y colaborativo. Salario competitivo, con un rango anual de 80,000 a 100,000 €, dependiendo de la experiencia y habilidades del candidato. Si estás emocionado/a por contribuir a proyectos desafiantes y trabajar en un entorno que fomente la innovación, ¡esperamos recibir tu solicitud! Envía tu currículum y carta de presentación.",
                               [{"ubicacion": "Madrid",
                                 "salario": "80,000 - 100,000 €"}],
                              ),
                             ("perfil buscado hombre/mujerel candidato seleccionado se ocupara de-desarrollar nuevas funcionalidades en la aplicacion-colaborar con un equipo multidisciplinar-aportar conocimientos en desarrollo android ya sea en kotlin o java-integracion de las aplicaciones-proponer mejoras en las aplicaciones y fomentar el uso de buenas practicas importante empresa de tecnologia busca un desarrollador androidgrandes posibilidades de crecimiento profesionallos conocimientos requeridos son-no se requiere experiencia previa idealmente 6 meses/1 año-conocer los lenguajes kotlin y/o java-formacion de grado universitario o fp2 en tecnologia-proactividad ganas de aprender-dinamismo y de crecer en un ambiente de alto rendimientoempresa puntera de tecnologia busca incorporar a su equipo de desarrollo un android developerse ofrece-salario competitivo negociable en funcion de experiencia y perfil-formacion continua y grandes posibilidades de crecimiento profesional-formar parte de una empresa puntera en tecnologia con una politica de crecimiento muy atractiva para sus empleados-participar en proyectos punteros en tecnologia-formar parte de un equipo altamente cualificado y multidisciplinar",
                               [{"ubicacion": "No especifica",
                                 "salario": "No especifica"}],
                              ),
                             ("desarrollo aplicaciones ios considerara plus- actitud trabajo equiposalariodependiendo experiencia formacion tecnica ofrecemos rango salarial 24-35 klo ofrecemos- empleo tiempo completo- contrato trabajo indefinido- proyectos innovadores grandes oportunidades formacion continua- oportunidad crecimiento profesional- posibilidad trabajo 100 remoto modalidad hibrida españa elegir- oficina ubicacion centrica distrito 56salamanca- buena conciliacion vida laboral familiar- horario trabajo flexible elegir- bebidas gratuitas oficina cafe agua infusiones zumos- servicio fruta semanal casa ametller oficina- snacks disponibles oficina- beneficios empleados colaboracion banco sabadell",
                               [{"ubicacion": "Salamanca",
                                 "salario": "24,000 - 35,000 €"}],
                              )],
                   )

In [283]:
# Create an extraction chain with the language model and job schema:
chain = create_extraction_chain(llm, job_schema)

In [342]:
for idx,row in df_datos_finales.iterrows():
    ubicacion = row["ubicacion"]
    salario = row["salario"]
    
    # Compruebo si las columnas ubicacion y salario contienen nan:
    if pd.isna(ubicacion) or pd.isna(salario):
        descripcion = row["descripcion"]
        
        # Si la descripcion contiene un número la tokenizamos:
        if re.search(r'\d', descripcion):
            clean_tokens = []
            
            # Detecto idioma:
            idioma = detect(descripcion)
            if idioma == "es":
                tokens = nltk.word_tokenize(text = descripcion, language = "spanish")
                stopwords = nltk.corpus.stopwords.words("spanish")
            elif idioma == "en":
                tokens = nltk.word_tokenize(text = descripcion, language = "english")
                stopwords = nltk.corpus.stopwords.words("english")
                
            # Quitamos stopwords:
            for token in tokens: 
                if token not in stopwords:
                    clean_tokens.append(token)
                    
            descripcion_sin_stopwords = " ".join(clean_tokens)
            
            # Saco la información para llenar los valores faltantes:
            response = chain.predict(text=descripcion_sin_stopwords)
            output = (response["data"]["Job offer"][0])
            
            if pd.isna(ubicacion):
                df_datos_finales.loc[idx, "ubicacion"] = output["ubicacion"]
                
            elif pd.isna(salario):
                df_datos_finales.loc[idx, "salario"] = output["salario"]
 
    break

In [343]:
df_datos_finales.loc[0]

titulo                                                    developer
empresa                                               almato iberia
fecha                                                    2024-01-24
herramientas                                  android, ios, flutter
descripcion       estamos buscando un programador android con al...
ubicacion                                                 Barcelona
presencialidad                                               remoto
funciones                                                  analista
jornada                                            jornada completa
experiencia                                                       3
tipo_contrato                                            indefinido
salario                                     24000 -36000  bruto/año
beneficios                                                      NaN
solicitudes                                                       0
fecha_scrapeo                                   

In [341]:
df_datos_finales.loc[0]

titulo                                                    developer
empresa                                               almato iberia
fecha                                                    2024-01-24
herramientas                                  android, ios, flutter
descripcion       estamos buscando un programador android con al...
ubicacion                                                       NaN
presencialidad                                               remoto
funciones                                                  analista
jornada                                            jornada completa
experiencia                                                       3
tipo_contrato                                            indefinido
salario                                     24000 -36000  bruto/año
beneficios                                                      NaN
solicitudes                                                       0
fecha_scrapeo                                   

# PRUEBAS OPENAI API

In [16]:
chat = ChatOpenAI(temperature=0, openai_api_key= openai_api_key)

In [47]:
def simplificador(oferta):

    respuesta = chat.invoke(
        [
            SystemMessage(content="clean the title job offer using two or three words maximum, don't put complementary information."),
            HumanMessage(content= oferta)
        ]
    )
    
    return respuesta.content

In [101]:
df_datos_finales.loc[478]

titulo                                         qa back - 100 remoto
empresa                                                michael page
fecha                                                    2024-01-04
herramientas                                           qa bitbucket
descripcion       perfil buscado hombre/mujer- diseñar planifica...
ubicacion                                                       NaN
presencialidad                                      no especificado
funciones                                                    tester
jornada                                            jornada completa
experiencia                                                       3
tipo_contrato                                            indefinido
salario                                     33000 -42000  bruto/año
beneficios                                                      NaN
solicitudes                                                       0
fecha_scrapeo                                   

In [48]:
df_datos_finales["titulo"][150:160].apply(simplificador)

150                  Data Analyst
151                       Auditor
152           Executive Associate
153    Project Manager Automotive
154            Frontend Developer
155     Contabilidad IFRS9/IFRS17
156              Business Analyst
157            Functional Manager
158              Support Engineer
159                 Fleet Manager
Name: titulo, dtype: object

In [45]:
df_datos_finales["titulo"][150:160]

150                         reliability data analyst
151                                 internal auditor
152            rev-advance executive associate track
153               project manager  sector automocion
154                 frontend developer accessibility
155    contabilidad ifrs9/ifrs17 banco catalan lider
156    business analyst - esg at wavespace ai center
157                    manager functional consulting
158                       associate support engineer
159                                  gestor de flota
Name: titulo, dtype: object

## LLM prueba

In [7]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", openai_api_key= openai_api_key)

C:\Users\regue\conda_ENV\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [8]:
def categorizer(oferta):
    
    template = """Eres un reclutador experimentado que tiene que simplificar el nombre de la oferta de trabajo, 
    dando nombre y rango del puesto, eliminando cualquier información complementaria al puesto como tecnologías, 
    programas, lenguajes de programacion, forma de trabajo(oficina, remoto, hibrido), simbolos, ..."

    """

    prompt = PromptTemplate(
        input_variables=["oferta"],
        template=template,
    )

    final_prompt = prompt.format(oferta= oferta)

    return llm(final_prompt).strip()

# Prueba limpieza descripción:

In [46]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-instruct",
    temperature=0,
    max_tokens=2000,
    openai_api_key=openai_api_key
)

In [58]:
job_schema = Object(id="Puesto de trabajo",
                    description="Información sobre el puesto de trabajo ofertado.",

                    # Notice I put multiple fields to pull out different attributes
                    attributes=[Text(id="herramientas", description="Lenguajes de programación y herramientas demandadas para el puesto de trabajo."),
                                Text(id="ubicacion", description="Localización del puesto de trabajo ofertado."),
                                Text(id="funciones", description="Funciones a desempeñar en el puesto de trabajo ofertado."),
                                Text(id="beneficios", description="Beneficios adicionales ofrecidos por la empresa."),
                                Number(id="salario",description="El salario o rango salarial ofrecido para el puesto de trabajo.")
                                ],
                    
                    examples=[('perfil buscado hombre/mujer- diseñar planificar especificar y ejecutar pruebas automatizadas back karate- diseñar planificar especificar y ejecutar pruebas manuales- participar en el chapterqa- pruebas en el sprint manual/automatizacion consultora tecnologicatrayectoria profesional- experiencia en pruebas manuales y automatizadas- experiencia bitbucket- open api- sonar appium- karate- git- cypress- bddpage consulting tech solutions es el area de pagegroup especializada en la realizacion de proyectos it en clientes ofreciendo una amplia variedad de soluciones tecnologicas a medida con la flexibilidad y agilidad requerida  contrato indefinido con pageconsulting tech solutions banda salarial 35k-42k oficina en Madrid formacion continua para desarrollar tu talento beneficios: seguro medico, ticket transporte, ticket restaurante, ticket guarderia. Plan de carrera en un ecosistema de talento tecnologico',
                               [{"herramientas": "bitbucket, git, openapi, karate, cypress",
                                 "ubicacion": "Madrid",
                                 "funciones": "tester",
                                 "beneficios": "seguro medico, ticket transporte, ticket restaurante, ticket guarderia",
                                 "salario": "35000 - 42000 €"}],
                              )]
                   )

In [62]:
for idx, row in df_datos_finales.iterrows():
    
    for col, value in row.items():
        if pd.isna(value):
            print(col)
    break

funciones
salario
beneficios


In [15]:
from langchain.chains import LLMChain


In [63]:
# Assuming 'llm' is your language model and 'job_schema' is your schema
text = df_datos_finales.loc[4321]["descripcion"]

# Create an extraction chain with the language model and job schema
chain = create_extraction_chain(llm, job_schema)

# Use the 'predict' method instead of 'predict_and_parse'
response = chain.predict(text=text)

# Extract data using the 'extract' method
output = (response["data"]["Puesto de trabajo"][0])

# Print the output
output

{'herramientas': 'co-simulation, cloud computing, machine learning, python, c',
 'ubicacion': 'not specified',
 'funciones': 'mdao co-simulation specialist',
 'beneficios': 'not specified',
 'salario': 'not specified'}

In [55]:
df_datos_finales.loc[1264]

titulo                                           network technician
empresa                                  alcorce telecomunicaciones
fecha                                                    2023-12-21
herramientas                                        redes, lan, wan
descripcion       debido a la expansion de alcorce telecomunicac...
ubicacion                                                       NaN
presencialidad                                      no especificado
funciones                                                     redes
jornada                                            jornada completa
experiencia                                                       3
tipo_contrato                                            indefinido
salario                                                         NaN
beneficios                                                      NaN
solicitudes                                                       0
fecha_scrapeo                                   

In [26]:
df_datos_finales["herramientas"][3478]

'rest maven spring hibernate web services spring boot junit microservicios'

In [59]:
for idx, row in df_datos_finales.iterrows():
    
    print(row)
    break

titulo                    lead data scientist - hybrid intelligence
empresa                                       capgemini engineering
fecha                                                    2024-01-04
herramientas      aptitudes para hacer presentaciones comunicaci...
descripcion       acerca del empleohybrid intelligence a capgemi...
ubicacion                                barcelona cataluña españa 
presencialidad                                              hibrido
funciones                                                       NaN
jornada                                            jornada completa
experiencia                                                       0
tipo_contrato                                       no especificado
salario                                                         NaN
beneficios                                                      NaN
solicitudes                                                      52
fecha_scrapeo                                   

"['aws glue', 'aws lambda', 'aws sagemaker', 'amazon web services (aws)', 'ciencia de datos', 'pegamento', 'python', 'r (lenguaje de programación)']"

# Limpieza herramientas --- FUNCIONA MAH O MENOH

In [15]:
# To help construct our Chat Messages
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate

# We will be using a chat model, defaults to gpt-3.5-turbo
from langchain_openai import ChatOpenAI

# To parse outputs and get structured data back
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

chat_model = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo', openai_api_key=openai_api_key)

In [37]:
instructions = """

"""

job_offer = df_datos_finales["herramientas"][2]

In [38]:
# Make your prompt which combines the instructions w/ the fruit names
prompt = (instructions + job_offer)

# Call the LLM
output = chat_model([HumanMessage(content=prompt)])

print (output.content)
print (type(output.content))

['svn', 'ccs', 's32ds', 'canoe', 'jira', 'parasoft']
<class 'str'>


In [34]:
df_datos_finales["herramientas"][2]

"['airflow', 'análisis de clústers', 'análisis predictivo', 'aprendizaje automático', 'aprendizaje profundo', 'ciencia de datos', 'procesamiento de lenguaje natural', 'python', 'reconocimiento de patrones', 'scala']"